In [17]:
""""
寫入 TFRecord 檔案的標準做法是：
step 1. 把所有資料轉換成「tf.train.Feature」格式。
step 2. 把所有的「tf.train.Feature」包裝成「tf.train.Features」格式。
step 3. 把所有的「tf.train.Features」組合成「tf.train.Example」格式。
step 4. 利用「tf.python_io.TFRecordWriter」將「tf.train.Example」寫入成 TFRecord 檔案。"""
import os
import tensorflow as tf
from PIL import Image
import numpy as np

In [21]:
"""
1.基礎數據格式
三種基礎數據類型：bytes，float，int64

對應tf.train中三種類型：BytesList (字符串列表), FloatList (浮點數列表), Int64List (64位整數列表)，構造它們三個依照下面的方式傳入相對應的value。

tf.train.BytesList(value=[context_idxs.tostring()]
tf.train.Int64List(value=[1,2])
tf.train.FloatList(value=[0.1,0.2])
2.example協議格式
要將我們的數據寫入.tfrecords 文件，需要將每一個樣本數據封裝為tf.train.Example格式，再將Example逐個寫入文件。Example格式中的數據基礎類型是tf.train.Feature

tf.train.Feature(): 它的參數是BytesList, FloatList, Int64List三種
---------------------
tf.train.Feature(bytes_list=tf.train.BytesList(value=)
tf.train.Feature(int64_list=tf.train.Int64List(value=)
tf.train.Feature(float_list=tf.train.FloatList(value=)

tf.train.Features: 它的參數是一個字典，kv對中v 的類型是Feature，對應每一個字段。
---------------------
tf.train.Features(feature={
            "k1": tf.train.Feature(bytes_list=tf.train.BytesList(value=])),
            "k2": tf.train.Feature(bytes_list=tf.train.BytesList(value=)),
            "k3": tf.train.Feature(float_list=tf.train.FloatList(value=)),
        })
        
在這裡,要將影像儲存為二進制資料, 影像標籤儲存成整數資料

"""
def _int64_feature(value):
    return tf.train.Feature(int64_list = tf.train.Int64List(value = [value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [value]))

In [22]:
#設定影像標籤,標籤名稱與對應子目錄相同
folder = 'C:/Users/User/Desktop/image/'
label = {'alphabat','brain'} 


In [19]:
#通過writer來寫TFRecord文件
writer = tf.python_io.TFRecordWriter(folder+'alphabat_brain.tfrecords')


In [20]:
#紀錄影像個數,使用enumerate遍歷label
count = 0  #紀錄影像數,方便確認
for index, name in enumerate(label):
    folder_path = folder +name +'/'
    #os.listdir() 方法用於返回指定的文件夾包含的文件或文件夾的名字的列表。這個列表以字母順序
    for img_name in os.listdir(folder_path): 
        img_path = folder_path + img_name #每一張影像的完整路徑
        img = Image.open(img_path)##開檔
        img = img.resize((128,128))
        img_raw = img.tobytes()#將影像轉為二進制
        example = tf.train.Example(
            features = tf.train.Features(
                feature = {
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value=[index])),
            'img_raw':tf.train.Feature(bytes_list = tf.train.BytesList(value = [img_raw]))
        })) #example對象對label,image數據進行封裝
        writer.write(example.SerializeToString())
        count = count + 1
writer.close()
print(count)    




9
